In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

In [23]:
project_root = os.path.abspath('..')

# Adding the project root to my system path
if project_root not in sys.path:
    sys.path.append(project_root)

from src.data_loader import telco_data_loader
from src.data_processor import telcoDataCleaner

In [24]:
# running the loader
raw_telco_data = telco_data_loader()

cleaner = telcoDataCleaner()

cleaned_data = cleaner.clean_data(raw_telco_data)

src.data_loader - Starting download of telco data
src.data_loader - Telco data download successful
src.data_loader - CSV data downloaded successfully
src.data_processor - Binary columns converted
src.data_processor - Converting gender
src.data_processor - Data types successfully converted
src.data_processor - Outlier Summary: {}
src.data_processor - Columns with missing values: {'OnlineSecurity': np.int64(1526), 'DeviceProtection': np.int64(1526), 'TechSupport': np.int64(1526), 'StreamingTV': np.int64(1526), 'StreamingMovies': np.int64(1526), 'TotalCharges': np.int64(11)}
src.data_processor - Filled missing values in TotalCharges with medians
src.data_processor - Filled missing values in Churn with medians


In [22]:
cleaned_data['clean_table']

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,0,1,0,1,0,No phone service,DSL,0.0,...,0.0,0.0,0.0,0.0,Month-to-month,Yes,Electronic check,29.85,29.85,0
1,5575-GNVDE,1,0,0,0,34,1,No,DSL,1.0,...,1.0,0.0,0.0,0.0,One year,No,Mailed check,56.95,1889.50,0
2,3668-QPYBK,1,0,0,0,2,1,No,DSL,1.0,...,0.0,0.0,0.0,0.0,Month-to-month,Yes,Mailed check,53.85,108.15,1
3,7795-CFOCW,1,0,0,0,45,0,No phone service,DSL,1.0,...,1.0,1.0,0.0,0.0,One year,No,Bank transfer (automatic),42.30,1840.75,0
4,9237-HQITU,0,0,0,0,2,1,No,Fiber optic,0.0,...,0.0,0.0,0.0,0.0,Month-to-month,Yes,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,1,0,1,1,24,1,Yes,DSL,1.0,...,1.0,1.0,1.0,1.0,One year,Yes,Mailed check,84.80,1990.50,0
7039,2234-XADUH,0,0,1,1,72,1,Yes,Fiber optic,0.0,...,1.0,0.0,1.0,1.0,One year,Yes,Credit card (automatic),103.20,7362.90,0
7040,4801-JZAZL,0,0,1,1,11,0,No phone service,DSL,1.0,...,0.0,0.0,0.0,0.0,Month-to-month,Yes,Electronic check,29.60,346.45,0
7041,8361-LTMKD,1,1,1,0,4,1,Yes,Fiber optic,0.0,...,0.0,0.0,0.0,0.0,Month-to-month,Yes,Mailed check,74.40,306.60,1
